In [1]:
import pandas as pd 
import nfl_data_py as nfl

In [2]:
pd.set_option('display.max_columns', None)  # Keine Begrenzung für Spaltenanzahl
pd.set_option('display.width', None)       # Keine Begrenzung der Breite

In [59]:
from data_loading import load_data, prepare_features, prepare_output

In [ ]:
df_schedule = nfl.import_schedules([2024])

df = load_data()
df_merged = prepare_output(df)
df_merged = df_merged[df_merged['season']==2024]


df_merged = df_merged.sort_values(['player_id', 'season', 'week'])
df_merged['did_play'] = 1

player_seasons = df_merged[['player_id', 'player_display_name', 'recent_team', 'season']].drop_duplicates()

all_weeks = []

for _, row in player_seasons.iterrows():
    season_weeks = 18 if row['season'] >= 2021 else 17
    weeks = pd.DataFrame({
        'player_id': row['player_id'],
        'player_display_name': row['player_display_name'],
        'recent_team': row['recent_team'],
        'season': row['season'],
        'week': range(1, season_weeks + 1),
    })
    all_weeks.append(weeks)
    
complete_weeks = pd.concat(all_weeks, ignore_index=True)

df_merged = pd.merge(
    complete_weeks,
    df_merged,
    on=['player_id', 'player_display_name','recent_team','season', 'week'],
    how='left'
)

df_merged['did_play'] = df_merged['did_play'].fillna(0)

# Liste aller Teams erstellen (unique Teams)
all_teams = pd.concat([df_schedule['home_team'], df_schedule['away_team']]).unique()

# Liste von Bye Weeks erstellen
bye_weeks = []

# Iteriere über jede Saison und jede Woche
for season in df_schedule['season'].unique():
    
    season_weeks = 18 if season >= 2021 else 17
    for week in range(1, season_weeks + 1):
        # Alle Spiele in dieser Saison und Woche filtern
        games_in_week = df_schedule[(df_schedule['season'] == season) & (df_schedule['week'] == week)]

        # Liste der Teams, die spielen
        teams_playing = set(games_in_week['home_team']).union(set(games_in_week['away_team']))

        # Berechne die Teams, die nicht spielen (also die Bye Week haben)
        teams_with_bye_week = set(all_teams) - teams_playing

        # Speichern der Teams mit Bye Week und der jeweiligen Saison und Woche
        for team in teams_with_bye_week:
            bye_weeks.append({'recent_team': team, 'season': season, 'week': week, 'opponent_team': 'BYE'})

        # Für die Teams, die gespielt haben, den Gegner speichern
        for _, game in games_in_week.iterrows():
            home_team = game['home_team']
            away_team = game['away_team']
            
            # Für jedes Team das gegnerische Team bestimmen
            bye_weeks.append({'recent_team': home_team, 'season': season, 'week': week, 'opponent_team': away_team})
            bye_weeks.append({'recent_team': away_team, 'season': season, 'week': week, 'opponent_team': home_team})

# DataFrame für Bye Weeks erstellen
df_bye = pd.DataFrame(bye_weeks)

df_merged = pd.merge(df_merged, df_bye, on=['recent_team', 'season', 'week'], how= 'left')
df_merged = df_merged.rename(columns={'opponent_team_y': 'opponent_team'})
df_merged = df_merged.drop(columns=['opponent_team_x'])

Downcasting floats.


/Users/simonmuhlbauer/Desktop/Uni/Master/WiSe_24-25/DSAI/Projekt/ff_data_science/src/data_loading.py:157: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: x.shift(1).ewm(span=5, min_periods=5).mean())
/Users/simonmuhlbauer/Desktop/Uni/Master/WiSe_24-25/DSAI/Projekt/ff_data_science/src/data_loading.py:171: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior

In [61]:
df_merged

,player_id,player_display_name,recent_team,season,week,position,depth_team,status,fantasy_points,did_play,opponent_team
0,23459,Aaron Rodgers,NYJ,2024,1,QB,1.0,ACT,8.580000,1.0,SF
1,23459,Aaron Rodgers,NYJ,2024,2,QB,1.0,ACT,15.140000,1.0,TEN
2,23459,Aaron Rodgers,NYJ,2024,3,QB,1.0,ACT,21.040001,1.0,NE
3,23459,Aaron Rodgers,NYJ,2024,4,QB,1.0,ACT,11.600000,1.0,DEN
4,23459,Aaron Rodgers,NYJ,2024,5,QB,1.0,ACT,11.760000,1.0,MIN
...,...,...,...,...,...,...,...,...,...,...,...
8869,39921,Trey Benson,ARI,2024,14,RB,2.0,ACT,1.900000,1.0,SEA
8870,39921,Trey Benson,ARI,2024,15,RB,2.0,ACT,2.200000,1.0,NE
8871,39921,Trey Benson,ARI,2024,16,NaN,NaN,NaN,NaN,0.0,CAR
8872,39921,Trey Benson,ARI,2024,17,NaN,NaN,NaN,NaN,0.0,LA


In [3]:
from data_loading_continous_playing import load_data, prepare_features, prepare_output

In [4]:
df = load_data()
df_feats = prepare_features(df)
df_out = prepare_output(df)

Downcasting floats.


/Users/simonmuhlbauer/Desktop/Uni/Master/WiSe_24-25/DSAI/Projekt/ff_data_science/src/data_loading_continous_playing.py:227: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: x.shift(1).ewm(span=5, min_periods=5).mean())
/Users/simonmuhlbauer/Desktop/Uni/Master/WiSe_24-25/DSAI/Projekt/ff_data_science/src/data_loading_continous_playing.py:241: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=

TypeError: unsupported operand type(s) for |: 'float' and 'bool'

In [5]:
df_schedule=nfl.import_schedules([2024])

In [6]:
df_schedule.head()

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,old_game_id,gsis,nfl_detail_id,pfr,pff,espn,ftn,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
6706,2024_01_BAL_KC,2024,REG,1,2024-09-05,Thursday,20:20,BAL,20.0,KC,27.0,Home,7.0,47.0,0.0,2024090500,59508.0,NaN,202409050kan,NaN,401671789.0,6449.0,7,7,124.0,-148.0,3.0,-118.0,-102.0,46.0,-110.0,-110.0,0,outdoors,grass,67.0,8.0,00-0034796,00-0033873,Lamar Jackson,Patrick Mahomes,John Harbaugh,Andy Reid,Shawn Hochuli,KAN00,GEHA Field at Arrowhead Stadium
6707,2024_01_GB_PHI,2024,REG,1,2024-09-06,Friday,20:15,GB,29.0,PHI,34.0,Neutral,5.0,63.0,0.0,2024090600,59509.0,NaN,202409060phi,NaN,401671805.0,6450.0,7,7,110.0,-130.0,2.0,-110.0,-110.0,49.5,-112.0,-108.0,0,outdoors,NaN,NaN,NaN,00-0036264,00-0036389,Jordan Love,Jalen Hurts,Matt LaFleur,Nick Sirianni,Ron Torbert,SAO00,Arena Corinthians
6708,2024_01_PIT_ATL,2024,REG,1,2024-09-08,Sunday,13:00,PIT,18.0,ATL,10.0,Home,-8.0,28.0,0.0,2024090800,59510.0,NaN,202409080atl,NaN,401671744.0,6451.0,7,7,160.0,-192.0,4.0,-110.0,-110.0,43.0,-115.0,-105.0,0,closed,fieldturf,NaN,NaN,00-0036945,00-0029604,Justin Fields,Kirk Cousins,Mike Tomlin,Raheem Morris,Brad Rogers,ATL97,Mercedes-Benz Stadium
6709,2024_01_ARI_BUF,2024,REG,1,2024-09-08,Sunday,13:00,ARI,28.0,BUF,34.0,Home,6.0,62.0,0.0,2024090801,59511.0,NaN,202409080buf,NaN,401671617.0,6452.0,7,7,250.0,-310.0,6.5,-105.0,-115.0,46.0,-112.0,-108.0,0,outdoors,a_turf,61.0,20.0,00-0035228,00-0034857,Kyler Murray,Josh Allen,Jonathan Gannon,Sean McDermott,Tra Blake,BUF00,New Era Field
6710,2024_01_TEN_CHI,2024,REG,1,2024-09-08,Sunday,13:00,TEN,17.0,CHI,24.0,Home,7.0,41.0,0.0,2024090802,59512.0,NaN,202409080chi,NaN,401671719.0,6453.0,7,7,164.0,-198.0,4.0,-108.0,-112.0,43.0,-110.0,-110.0,0,outdoors,grass,67.0,8.0,00-0039152,00-0039918,Will Levis,Caleb Williams,Brian Callahan,Matt Eberflus,Shawn Smith,CHI98,Soldier Field
